In [5]:
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import urllib
import os, io
import  pandas  as pd
import logging

#logging.basicConfig(level=logging.INFO)
log=logging.getLogger(__name__)
consoleHandler = logging.StreamHandler()
consoleHandler.setLevel(logging.DEBUG)
log.addHandler(consoleHandler)

import requests
from minio import Minio
load_dotenv()
S3_ACCESS_KEY = os.getenv("S3_ACCESS_KEY")
MINIO_SECRET_KEY = os.getenv("S3_SECRET_KEY")
MINIO_ENDPOINT = os.getenv("S3_ENDPOINT")
MINIO_BUCKET = "resilentpublic"
S3_PREFIX = "data/cdc/wonder/raw/{year}/{week}/"
client = Minio(MINIO_ENDPOINT, S3_ACCESS_KEY, MINIO_SECRET_KEY)
wonder_template_url = "https://wonder.cdc.gov/nndss/nndss_weekly_tables_menu.asp?mmwr_year={year}&mmwr_week={week}"
wonder_base_url = "https://wonder.cdc.gov"; 
# 2024 <a href="/nndss/static/2024/37/2024-37-table10.txt" download="">Export Data</a>
# 1998 <a href="/nndss/static/2024/37/2024-37-table10.txt" download="">Export Data</a>
wonder_export_dom="Export Data"

In [6]:
#years=[x for x in range(1996,2024)]
years=[x for x in range(2024,2025)]
weeks=[x for x in range(1,52)]
#weeks =[1]
#years=[2024]
final_year=2025
final_week=5

In [7]:
# what files do we have
with requests.session() as s:
    for year in years:
        for week in weeks:
            if year >= final_year and week > final_week:
                continue
            url = wonder_template_url.format(year=year, week=week)
            weekstr = str(week).zfill(2)
            url = wonder_template_url.format(year=year, week=weekstr)
            links=[]
            with s.get(url, stream=True) as response:
                soup = BeautifulSoup(response.content, 'html.parser')
                for link in soup.find_all('a', href=True):
                    if "export data" in link.text.lower():
                        links.append(link['href'])
                # write out weeks data to bucket
                response.close()
            for download_link in links:
                log.info(download_link)
                if download_link.startswith('/'):
                    d_url = wonder_base_url + download_link
                    paths = download_link.split('/')[-1]           
                    s3_path =  S3_PREFIX.format(year=year, week=weekstr) + paths
                    log.info(s3_path)
                else:
                    d_url= wonder_base_url +"/nndss/"+download_link
                    qs = urllib.parse.parse_qs(download_link)
                    paths = f"table_{qs['mmwr_table'][0]}"
                    s3_path =  S3_PREFIX.format(year=year, week=weekstr) + paths
                    log.info(s3_path)

In [10]:
with requests.session() as s:
    for year in years:
        for week in weeks:
            if year >= final_year and week > final_week:
                continue
            weekstr = str(week).zfill(2)
            url = wonder_template_url.format(year=year, week=weekstr)
            links=[]
            with s.get(url) as response:
                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, 'html.parser')
                    for link in soup.find_all('a', href=True):
                        if "export data" in link.text.lower():
                            links.append(link['href'])
                else:
                    log.info(f"{response.status_code} {url}")
                response.close()
                soup = None
            for download_link in links:
                log.info(download_link)
                if download_link.startswith('/'):
                    d_url = wonder_base_url + download_link
                    paths = download_link.split('/')[-1]           
                    s3_path =  S3_PREFIX.format(year=year, week=weekstr) + paths
                    log.info(s3_path)
                else:
                    d_url= wonder_base_url +"/nndss/"+download_link
                    qs = urllib.parse.parse_qs(download_link)
                    paths = f"table_{qs['mmwr_table'][0]}"
                    s3_path =  S3_PREFIX.format(year=year, week=weekstr) + paths
                    log.info(s3_path)  
                with s.get(d_url, stream=False) as rf:
                    if rf.status_code == 200:
                        content = rf.text
                        rf.close()
                        metadata={"export_link": download_link, "url": d_url,
                                          "source_page_url": url, }
                        
                        result = client.put_object(
                            MINIO_BUCKET, s3_path,
                            data=io.BytesIO(content.encode('utf-8')), 
                            length=len(content),
                            content_type="text/plain", metadata=metadata
                         )
                        log.info(
                            "created {0} object; etag: {1}, version-id: {2}".format(
                                result.object_name, result.etag, result.version_id,
                            ),
                        )
                        content = None
                    else:
                        log.info(f"{response.status_code} {url} {response.reason}")

KeyboardInterrupt: 

[]